In [1]:
import pandas as pd
from pandas import Series,DataFrame
import seaborn as sns
import numpy as np

In [4]:
#데이터를 우선 로드해줍니다 
ratings = pd.read_csv("../data/RecommenderSystems_R-master/movie_rating.csv")

In [5]:
#데이터 세트의 차원을 확인하기 
print("Dimensions of rating: {}".format(ratings.shape))

Dimensions of rating: (31, 3)


In [6]:
#rating에는 레이팅을 한 사용자 그리고 영화 제목 rating 정보가 포함 되어 있습니다. 
ratings.head()

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
1,Jack Matthews,Snakes on a Plane,4.0
2,Jack Matthews,You Me and Dupree,3.5
3,Jack Matthews,Superman Returns,5.0
4,Jack Matthews,The Night Listener,3.0


In [8]:
#추천엔진을 구축하는데 유용한 형식으로 변경을 시도해줍니다 
#r에서는 reshpae 을 사용해 주었습니다  python에서는 pivot table을 사용해줄수있습니다 
#데이터 재구조화 
movie_rating= ratings.pivot(index='title', columns='critic',values='rating')
movie_rating = movie_rating.reset_index()
movie_rating

critic,title,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
0,Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
1,Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
2,Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
3,Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
4,The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN
5,You Me and Dupree,2.5,3.5,3.5,2.5,2.0,1.0


In [12]:
#이제 사용자 사이의 유사도를 구해줍니다 
#python에서는 corr()함수를 사용해 줄수있습니다 
sim_users = movie_rating.corr()
sim_users = sim_users.reset_index()
sim_users

#mike 와 Toby 또는 Lisa와 Toby가 유사한 취향을 가지고 있는 걸 확인할수있습니다. 

critic,critic,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
0,Claudia Puig,1.000000,0.314970,0.028571,0.566947,0.566947,0.893405
1,Gene Seymour,0.314970,1.000000,0.963796,0.396059,0.411765,0.381246
2,Jack Matthews,0.028571,0.963796,1.000000,0.747018,0.211289,0.662849
3,Lisa Rose,0.566947,0.396059,0.747018,1.000000,0.594089,0.991241
4,Mick LaSalle,0.566947,0.411765,0.211289,0.594089,1.000000,0.924473
5,Toby,0.893405,0.381246,0.662849,0.991241,0.924473,1.000000


In [22]:
movie_rating.head()

critic,title,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Mick LaSalle,Toby
0,Just My Luck,3.0,1.5,NaN,3.0,2.0,NaN
1,Lady in the Water,NaN,3.0,3.0,2.5,3.0,NaN
2,Snakes on a Plane,3.5,3.5,4.0,3.5,4.0,4.5
3,Superman Returns,4.0,5.0,5.0,3.5,3.0,4.0
4,The Night Listener,4.5,3.0,3.0,3.0,3.0,NaN


In [23]:
#사용자의 등급예측 
#여기서는 Toby가 예측 하지 않은 영화에대해서 예상을 해보는걸 할것입니다. 
# 아직 토비가 예측 하지 않은 영화들을 찾아봅니다 
rating_critic = movie_rating.ix[:,['title','Toby']]
rating_critic

critic,title,Toby
0,Just My Luck,NaN
1,Lady in the Water,NaN
2,Snakes on a Plane,4.5
3,Superman Returns,4.0
4,The Night Listener,NaN
5,You Me and Dupree,1.0


In [26]:
#토비가 보지 않은 영화들 제목만 추출해 줍니다 
tiles_na_critic = pd.DataFrame(rating_critic[rating_critic['Toby'].isnull()]['title'])
tiles_na_critic

,title
0,Just My Luck
1,Lady in the Water
4,The Night Listener


In [39]:
#원본 데이터에서 Toby가 평가 하지 않은 항목들만 isin 함수를 사용해서 불러와 줍니다 
rating_t =  ratings[ratings['title'].isin(tiles_na_critic['title'])]
rating_t

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
4,Jack Matthews,The Night Listener,3.0
5,Mick LaSalle,Lady in the Water,3.0
7,Mick LaSalle,Just My Luck,2.0
10,Mick LaSalle,The Night Listener,3.0
12,Claudia Puig,Just My Luck,3.0
15,Claudia Puig,The Night Listener,4.5
16,Lisa Rose,Lady in the Water,2.5
18,Lisa Rose,Just My Luck,3.0
20,Lisa Rose,The Night Listener,3.0


In [40]:
#similarity를 merge해줍니다 
ratings_t = pd.merge(rating_t,sim_users.ix[:,['critic','Toby']], on='critic')
ratings_t

,critic,title,rating,Toby
0,Jack Matthews,Lady in the Water,3.0,0.662849
1,Jack Matthews,The Night Listener,3.0,0.662849
2,Mick LaSalle,Lady in the Water,3.0,0.924473
3,Mick LaSalle,Just My Luck,2.0,0.924473
4,Mick LaSalle,The Night Listener,3.0,0.924473
5,Claudia Puig,Just My Luck,3.0,0.893405
6,Claudia Puig,The Night Listener,4.5,0.893405
7,Lisa Rose,Lady in the Water,2.5,0.991241
8,Lisa Rose,Just My Luck,3.0,0.991241
9,Lisa Rose,The Night Listener,3.0,0.991241


In [43]:
#등급과 유사도 값을 곱한후 그결과 값을 새로은 변수인 sim_rating에추가해줍니다 
ratings_t["sim_rating"] = ratings_t['Toby'] * ratings_t['rating']
ratings_t

,critic,title,rating,Toby,sim_rating
0,Jack Matthews,Lady in the Water,3.0,0.662849,1.988547
1,Jack Matthews,The Night Listener,3.0,0.662849,1.988547
2,Mick LaSalle,Lady in the Water,3.0,0.924473,2.773420
3,Mick LaSalle,Just My Luck,2.0,0.924473,1.848947
4,Mick LaSalle,The Night Listener,3.0,0.924473,2.773420
5,Claudia Puig,Just My Luck,3.0,0.893405,2.680215
6,Claudia Puig,The Night Listener,4.5,0.893405,4.020323
7,Lisa Rose,Lady in the Water,2.5,0.991241,2.478102
8,Lisa Rose,Just My Luck,3.0,0.991241,2.973722
9,Lisa Rose,The Night Listener,3.0,0.991241,2.973722


In [47]:
#이전단계에서 구한 각영화의 모든 등급을 더한구 각 값을각 리뷰어의 유사도 값의 합으로 나눈다 
ratings_t.groupby('title').sim_rating.sum()/ratings_t.groupby('title').Toby.sum()

title
Just My Luck          2.530981
Lady in the Water     2.832550
The Night Listener    3.347790
dtype: float64

In [49]:
#이제 Toby가 평가한 세 영화의 mean값을 가져와 봅니다 
rating_critic[rating_critic['Toby'].notnull()].mean()

critic
Toby    3.166667
dtype: float64

In [ ]:
#여기서 Toby의 평균인 3.146667보다 높은  The Night Listener 를 평가 해줄수있습니다. 